In [18]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import copy
%matplotlib inline

In [19]:
telecom = pd.read_csv('telecom_churn_data.csv')

In [20]:
telecom.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [21]:
telecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [22]:
#As per problem statement creating a column of average recharge amount of first two months
telecom['avg_rech_amt'] = (telecom['total_rech_amt_6'] + telecom['total_rech_amt_7'])/2

##### As per problem statement creating dataframe which has only customers with average recharge amount more than or equal to 70th percentile of average recharge amount 


In [23]:
Q = telecom.avg_rech_amt.quantile(0.7)

In [24]:
telecom = telecom[telecom.avg_rech_amt > Q]

In [25]:
telecom.shape

(29979, 227)

In [26]:
telecom.nunique()

mobile_number           29979
circle_id                   1
loc_og_t2o_mou              1
std_og_t2o_mou              1
loc_ic_t2o_mou              1
last_date_of_month_6        1
last_date_of_month_7        1
last_date_of_month_8        1
last_date_of_month_9        1
arpu_6                  29230
arpu_7                  29228
arpu_8                  28376
arpu_9                  27300
onnet_mou_6             18805
onnet_mou_7             18933
onnet_mou_8             17597
onnet_mou_9             16666
offnet_mou_6            22440
offnet_mou_7            22638
offnet_mou_8            21499
offnet_mou_9            20437
roam_ic_mou_6            4337
roam_ic_mou_7            3648
roam_ic_mou_8            3653
roam_ic_mou_9            3368
roam_og_mou_6            5173
roam_og_mou_7            4430
roam_og_mou_8            4381
roam_og_mou_9            4002
loc_og_t2t_mou_6        11149
                        ...  
night_pck_user_6            2
night_pck_user_7            2
night_pck_

#### Here we can see that some columns have only 1 unique value. And there are few which have only 2,5 and 6. Let's remove these columns.

In [27]:
telecom = telecom.loc[:,telecom.apply(pd.Series.nunique) >6]

In [28]:
# Column with minimum variatio has following number of unique values
min(telecom.nunique())

11

In [29]:
#dropping columns which have more than 70% NaN values
# for cl in telecom.columns:
#     if round(100*(telecom[cl].isnull().sum()/len(telecom[cl].index)), 2) > 70:
#         telecom = telecom.drop(cl, axis=1)

In [30]:
# round(100*(telecom.isnull().sum()/len(telecom.index)), 2)

#### Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase. Therefore condition will be as follows: 


In [31]:
telecom['churn'] = np.where(((telecom['total_ic_mou_9']==0) & (telecom['total_og_mou_9']==0)) & ((telecom['vol_2g_mb_9']==0) & (telecom['vol_3g_mb_9']==0)),1,0)

In [32]:
telecom['churn'].value_counts()

0    27390
1     2589
Name: churn, dtype: int64

In [33]:
#dropping columns with '_9' at the end
new_col = [c for c in telecom.columns if c[-2:] != '_9']
telecom = telecom[new_col]

In [34]:
# Checking percentage missing values in each columns
(100*(telecom.isnull().sum()/len(telecom.index)))

mobile_number          0.000000
arpu_6                 0.000000
arpu_7                 0.000000
arpu_8                 0.000000
onnet_mou_6            1.054071
onnet_mou_7            1.010707
onnet_mou_8            3.128857
offnet_mou_6           1.054071
offnet_mou_7           1.010707
offnet_mou_8           3.128857
roam_ic_mou_6          1.054071
roam_ic_mou_7          1.010707
roam_ic_mou_8          3.128857
roam_og_mou_6          1.054071
roam_og_mou_7          1.010707
roam_og_mou_8          3.128857
loc_og_t2t_mou_6       1.054071
loc_og_t2t_mou_7       1.010707
loc_og_t2t_mou_8       3.128857
loc_og_t2m_mou_6       1.054071
loc_og_t2m_mou_7       1.010707
loc_og_t2m_mou_8       3.128857
loc_og_t2f_mou_6       1.054071
loc_og_t2f_mou_7       1.010707
loc_og_t2f_mou_8       3.128857
loc_og_t2c_mou_6       1.054071
loc_og_t2c_mou_7       1.010707
loc_og_t2c_mou_8       3.128857
loc_og_mou_6           1.054071
loc_og_mou_7           1.010707
                        ...    
av_rech_

In [35]:
#dropping columns which have more than 50% NaN values
for cl in telecom.columns:
    if round(100*(telecom[cl].isnull().sum()/len(telecom[cl].index)), 2) > 50:
        telecom = telecom.drop(cl, axis=1)

In [36]:
telecom= telecom.dropna(axis=0)

In [37]:
telecom.shape

(7524, 152)

In [38]:
# All of the object datatype columns are of dates. We are dropping those here
obj_col =telecom.select_dtypes(include=['object']).columns

In [39]:
telecom = telecom.drop(obj_col,axis=1)

In [40]:
X = telecom.drop(['mobile_number','churn'], axis=1)
y = telecom[['churn']]

In [41]:
print(X.shape)
print(y.shape)

(7524, 144)
(7524, 1)
